In [106]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [107]:
# Load the dataset using pandas 
df = pd.read_csv("Titanic-Dataset.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [109]:
# Remove Cabin because it has too many mssing values 
df.drop(['Cabin'], axis = 1, inplace = True)
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
715,716,0,3,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.0,0,0,348124,7.6500,S
832,833,0,3,"Saad, Mr. Amin",male,NaN,0,0,2671,7.2292,C
312,313,0,2,"Lahtinen, Mrs. William (Anna Sylfven)",female,26.0,1,1,250651,26.0000,S


In [110]:
# Lets, extract those columns which require preprocessing
df_extract = df.iloc[:, [1,4,5,10]]
df_extract.sample(3)

,Survived,Sex,Age,Embarked
367,1,female,NaN,C
796,1,female,49.0,S
267,1,male,25.0,S


In [111]:
X = df[['Sex', 'Age', 'Embarked']]
y = df[['Survived']]

# Train split test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [112]:
# One-Hot encoding Sex 
ohe_sex = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

X_train_sexEnc = ohe_sex.fit_transform(X_train[['Sex']])
X_test_sexEnc = ohe_sex.transform(X_test[['Sex']])

X_train_sexEnc.shape

(712, 1)

In [113]:
# Impute missing values in Age using simple imputer 
si_age = SimpleImputer(strategy='mean')

X_train_ageImp = si_age.fit_transform(X_train[['Age']])
X_test_ageImp = si_age.transform(X_test[['Age']])

X_train_ageImp.shape

(712, 1)

In [114]:
# Embarked require two preprocessing steps
# First impute missing values using simple imputer
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_embImp = si_embarked.fit_transform(X_train[['Embarked']])
X_test_embImp = si_embarked.transform(X_test[['Embarked']])

# 2nd after imputing missing values now encode Embarked using One-Hot encoder
ohe_embarked = OneHotEncoder(drop='first', handle_unknown='ignore')

X_train_embTra = ohe_embarked.fit_transform(X_train_embImp).toarray()
X_test_embTra = ohe_embarked.transform(X_test_embImp).toarray()

X_train_embTra.shape

(712, 2)

In [115]:
# Now my task is to combine all preprocessing steps into one array
X_train_transformed = np.concatenate([X_train_sexEnc, X_train_ageImp, X_train_embTra], axis=1)
X_test_transformed = np.concatenate([X_test_sexEnc, X_test_ageImp, X_test_embTra], axis=1)

X_train_transformed

array([[ 1. , 45.5,  0. ,  1. ],
       [ 1. , 23. ,  0. ,  1. ],
       [ 1. , 32. ,  0. ,  1. ],
       ...,
       [ 1. , 41. ,  0. ,  1. ],
       [ 0. , 14. ,  0. ,  1. ],
       [ 1. , 21. ,  0. ,  1. ]])

In [116]:
X_train_transformed.shape

(712, 4)

In [120]:
# Now converting this numpy array into pandas dataframe
columns = list(ohe_sex.get_feature_names_out(['Sex'])) + ['Age'] + list(ohe_embarked.get_feature_names_out(['Embarked']))


X_train_transformed = pd.DataFrame(X_train_transformed, columns=columns)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=columns)

X_train_transformed.head()

,Sex_male,Age,Embarked_Q,Embarked_S
0,1.0,45.5,0.0,1.0
1,1.0,23.0,0.0,1.0
2,1.0,32.0,0.0,1.0
3,1.0,26.0,0.0,1.0
4,0.0,6.0,0.0,1.0
